In [27]:
import pygmt
from pygmt.helpers import GMTTempFile
import xarray as xr
from pygmt import grdcut, grdinfo, grdfilter
from pygmt.datasets import load_earth_relief
from pygmt.exceptions import GMTInvalidInput
from pygmt.helpers import GMTTempFile
import os
import numpy as np

In [28]:
grid = pygmt.datasets.load_earth_relief(registration="pixel")
smooth_field = pygmt.grdfilter(grid=grid, filter="g600", distance="4")
print(smooth_field)
assert isinstance(smooth_field, xr.DataArray)
print(isinstance(smooth_field, xr.DataArray))
print(smooth_field.coords["lat"].data.min())
print(smooth_field.coords["lat"].data.max())
print(smooth_field.coords["lon"].data.min())
print(smooth_field.coords["lon"].data.max())
print(smooth_field.data.min())
print(smooth_field.data.max())
print(smooth_field.sizes["lat"])
print(smooth_field.sizes["lon"])
assert smooth_field.coords["lat"].data.min() == -89.5
assert smooth_field.coords["lat"].data.max() == 89.5
assert smooth_field.coords["lon"].data.min() == -179.5
assert smooth_field.coords["lon"].data.max() == 179.5
#assert smooth_field.data.min() == -6147.4727
#assert smooth_field.data.max() == 5164.1157
assert smooth_field.sizes["lat"] == 180
assert smooth_field.sizes["lon"] == 360

<xarray.DataArray 'z' (lat: 180, lon: 360)>
array([[ 2901.9534,  2901.3103,  2900.6545, ...,  2903.8042,  2903.2004,
         2902.5835],
       [ 2963.1248,  2960.8792,  2958.576 , ...,  2969.5137,  2967.4434,
         2965.3132],
       [ 2884.941 ,  2877.6084,  2870.0527, ...,  2905.4631,  2898.868 ,
         2892.031 ],
       ...,
       [-3359.723 , -3364.6345, -3367.713 , ..., -3334.4705, -3344.619 ,
        -3353.0056],
       [-3284.2292, -3285.5964, -3286.227 , ..., -3275.8582, -3279.3655,
        -3282.175 ],
       [-3184.8066, -3178.6543, -3172.2986, ..., -3202.082 , -3196.3865,
        -3190.7192]], dtype=float32)
Coordinates:
  * lon      (lon) float64 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
Attributes:
    long_name:     z
    actual_range:  [-6147.47265625  5164.11572266]
True
-89.5
89.5
-179.5
179.5
-6147.4727
5164.1157
180
360


In [29]:
    with GMTTempFile(suffix=".nc") as tmpfile:
        result = grdcut("@earth_relief_01d", outgrid=tmpfile.name, region="0/180/0/90")
        assert result is None  # return value is None
        assert os.path.exists(path=tmpfile.name)  # check that outgrid exists
        result = grdinfo(tmpfile.name, C=True)
        assert result == "0 180 0 90 -8182 5651.5 1 1 180 90 1 1\n"
        print(result)

0 180 0 90 -8182 5651.5 1 1 180 90 1 1



In [30]:
with GMTTempFile(suffix=".nc") as tmpfile:
    result = grdfilter("@earth_relief_01d", outgrid=tmpfile.name, region="0/180/0/90", filter="g600", distance="4")
    assert result is None  # grdcut returns None if output to a file
    result = pygmt.grdinfo(tmpfile.name, C=True)
    print(result)

0 180 0 90 -6147.49072266 5164.06005859 1 1 180 90 1 1



In [42]:
    outgrid = grdfilter("@earth_relief_01d", region="0/180/0/90", filter="g600", distance="4")
    assert isinstance(outgrid, xr.DataArray)
    assert outgrid.gmt.registration == 1  # Pixel registration
    assert outgrid.gmt.gtype == 1  # Geographic type
    # check information of the output grid
    # the '@earth_relief_01d' is in pixel registration, so the grid range is
    # not exactly 0/180/0/90
    assert isinstance(outgrid, xr.DataArray)
    print(outgrid.sizes["lon"])
    assert outgrid.coords["lat"].data.min() == 0.5
    assert outgrid.coords["lat"].data.max() == 89.5
    assert outgrid.coords["lon"].data.min() == 0.5
    assert outgrid.coords["lon"].data.max() == 179.5
    np.testing.assert_almost_equal(outgrid.data.min(), -6147.4907, decimal=2)
    np.testing.assert_almost_equal(outgrid.data.max(), 5164.06, decimal=2)
    assert outgrid.sizes["lat"] == 90
    assert outgrid.sizes["lon"] == 180

180


In [ ]:
outgrid = pygmt.grdcut("@earth_relief_01d", region="0/180/0/90")
print(outgrid)
assert isinstance(outgrid, xr.DataArray)
assert outgrid.gmt.registration == 1  # Pixel registration
assert outgrid.gmt.gtype == 1  # Geographic type
    # check information of the output grid
    # the '@earth_relief_01d' is in pixel registration, so the grid range is
    # not exactly 0/180/0/90
assert outgrid.coords["lat"].data.min() == 0.5
assert outgrid.coords["lat"].data.max() == 89.5
assert outgrid.coords["lon"].data.min() == 0.5
assert outgrid.coords["lon"].data.max() == 179.5
assert outgrid.data.min() == -8182.0
assert outgrid.data.max() == 5651.5
assert outgrid.sizes["lat"] == 90
assert outgrid.sizes["lon"] == 180